In [1]:
#------------------------------------------------------------------------------------
# Homework Part II - PyRamen
# -----------------------------------------------------------------------------------
import numpy_financial as npf
import csv
from pathlib import Path

#------------------------------------------------------------------------------------
# Function
#   name:   parse_menu_data
#   input:  Path
#   return: returns dictionary of lists using item name as key for fast lookup
# ------------------------------------------------------------------------------------
def parse_menu_data(path) :

    # Need fast lookup by key (item name)
    # Can index into list of props
    # Thus, we want dicitonary of lists
    menu_data = {}

    # Open file for read
    with open(path, 'r') as file :

        # Init csv reader
        csvreader = csv.reader(file, delimiter = ',')  

        # Parse header (item, category, description, price, cost)
        header = next(csvreader)

        # Iterate over rows
        for row in csvreader :

            # Parse key and list of details from line
            key = row[0]
            details = [ row[1], row[2], row[3], row[4] ]
            
            # Add dictionary entry
            menu_data[key] = details

    return menu_data

# ------------------------------------------------------------------------------------
# Function
#   name:   parse_sales_data
#   input:  Path
#   return: list of lists
# ------------------------------------------------------------------------------------
def parse_sales_data(path) :

    # 2D list
    sales_data = []

    # Open file for read
    with open(path, 'r') as file :

        # Init csv reader
        csvreader = csv.reader(file, delimiter = ',')  

        # Parse header (item, category, description, price, cost)
        header = next(csvreader)

        # Iterate over rows
        for row in csvreader :

            # Add sales record to list of sales data
            sales_record = [ row[0], row[1], row[2], row[3], row[4] ]
            sales_data.append(sales_record)

    return sales_data

# ------------------------------------------------------------------------------------
# Function
#   name:   write_to_file
#   input:  sales_report (dictionary), path
#   return: none
# ------------------------------------------------------------------------------------
def write_to_file(sales_report, path) :
    with open(path, 'w') as file :
        for key, value in sales_report.items() :
            file.write(f"{key} {value}\n")


# ------------------------------------------------------------------------------------

# Paths
menu_data_path = Path('menu_data.csv')
sales_data_path = Path('sales_data.csv')
results_path = Path('output.txt')

# Parse menu data
menu_data = parse_menu_data(menu_data_path)
sales_data = parse_sales_data(sales_data_path)

# Init new sales report
# This will be dictionary of dictionary items
sales_report = {}

# Errors collection
errors = []

# Iterate over sales records
for sale_item in sales_data :    
    
    # Parse sales report details
    key = sale_item[4]
    id = sale_item[0]
    date = sale_item[1]
    card_number = sale_item[2]
    qty = int(sale_item[3])

    # Look for matching key in menu
    # If not found, add to errors collection and skip loop iteration
    if (key not in menu_data) :
        errors.append(f"{key} is not on the menu!\n")
        continue
    
    # Lookup price and cost from menu data
    price = int(menu_data[key][2])
    cost = int(menu_data[key][3])

    # Calculate profit
    profit = (price - cost) * qty

    if (key not in sales_report) :

        # New entry
        sales_report_line = {
            "01-count" : qty,
            "02-revenue" : price * qty,
            "03-cogs" : cost * qty,
            "04-profit" : profit
        }

        # Add
        sales_report[key] = sales_report_line

    else :

        # Update existing entry
        sales_report[key]["01-count"] += qty
        sales_report[key]["02-revenue"] += (price * qty)
        sales_report[key]["03-cogs"] += (cost * qty)
        sales_report[key]["04-profit"] += profit

# Debug - verify item counts
print(f"--------------------------------------------")
print(f"Menu data item count: {len(menu_data)}")
print(f"Sales data item count: {len(sales_data)}")
print(f"Sales report item count: {len(sales_report)}")
print(f"Error count: {len(errors)}, {errors}")

# Write to file
write_to_file(sales_report, results_path)





--------------------------------------------
Menu data item count: 32
Sales data item count: 74124
Sales report item count: 11
Error count: 0, []
